# Beta Lag Research
## Import Packages

In [1]:
import datetime as dt
import pandas as pd
import numpy as np
from pandas_datareader import data
import statsmodels.formula.api as sm
import time
import wmcm

## Load Universe
I created a CSV listing all stocks included in SPDR’s sector funds as of January 24th, 2016. It is loaded as a pandas dataframe. In order to make calls to Yahoo, all periods are replaced with hyphens.

We also pull all sector tickers and SPY as a proxy for the market as a whole.

In [2]:
universe_stocks = pd.read_csv('inputs/stocks.csv', index_col='symbol')
universe_stocks.index = map(lambda x: x.replace('.', '-'), universe_stocks.index)

universe_sectors = pd.read_csv('inputs/sectors.csv', index_col='symbol')
universe_sectors.index = map(lambda x: x.replace('.', '-'), universe_sectors.index)

# limited to a few stocks
# universe_stocks = universe_stocks.ix[['GOOG', 'PM', 'XOM']]
universe_stocks.head()

,company_name,index_weight,sector,sector_name
PG,Procter & Gamble,11.90%,XLP,Consumper Staples
KO,Coca-Cola Co,9.31%,XLP,Consumper Staples
PM,Philip Morris International,7.61%,XLP,Consumper Staples
CVS,CVS Health Corporation,6.16%,XLP,Consumper Staples
MO,Altria Group Inc,5.84%,XLP,Consumper Staples


## Pull Price History

Price history is pulled from Yahoo into separate Stock instances. Stocks were pulled from January 1st 2010 through December 31st, 2015.

In [5]:
# stocks = {ticker : wmcm.Stock(ticker, interval = 'w') for ticker in universe_stocks.index}
stocks = {}
for ticker in universe_stocks.index:
    time.sleep(1)
    try:
        stocks[ticker] = wmcm.Stock(ticker, interval = 'w')
        if len(stocks[ticker].earnings_df)<1:
            del stocks[ticker]
    except ValueError:
        print('ValueError: {}'.format(ticker))
    except:
        print('OtherError: {}'.format(ticker))


market = wmcm.Market('SPY', interval = 'w')

ValueError: CCL


/Users/chriswhite1/Dropbox/White Morgan Capital Management/wmcm/wmcm/wmcm/core/stock.py:24: UserWarning: No Earnings Data found for FOX!
  warnings.warn("No Earnings Data found for {}!".format(self.ticker))
/Users/chriswhite1/Dropbox/White Morgan Capital Management/wmcm/wmcm/wmcm/core/stock.py:24: UserWarning: No Earnings Data found for DISCK!
  warnings.warn("No Earnings Data found for {}!".format(self.ticker))


ValueError: DISCA
ValueError: EOG
ValueError: BHI
ValueError: APA
ValueError: DVN
ValueError: OKE


/Users/chriswhite1/Dropbox/White Morgan Capital Management/wmcm/wmcm/wmcm/core/stock.py:24: UserWarning: No Earnings Data found for BRK-b!
  warnings.warn("No Earnings Data found for {}!".format(self.ticker))


OtherError: WLTW
ValueError: REGN
ValueError: GWW
ValueError: CHRW
ValueError: LYB
ValueError: CBG
ValueError: ADP
ValueError: PNW


## Calculate Betas on Full Sample

In [6]:
def make_analysis_df(tic, stocks, market):
    security_df = stocks[tic].adj_returns.loc[:, ['ret_cc', 'earnings_week']]
    security_df.columns = ['security_return', 'earnings_week']
    security_df['security_return_lag'] = security_df['security_return'].shift(1)
    security_df['earnings_week_lag'] = security_df['earnings_week'].shift(1)
    market_df = market.adj_returns.loc[:, 'ret_cc'].to_frame('market_return').reset_index()
    analysis_df = pd.merge(left = security_df, right = market_df, left_index = True, right_on = 'Date', how = 'left')
    analysis_df.set_index('Date', inplace = True)
    
    return analysis_df


make_analysis_df('GOOG', stocks, market).head()

,security_return,earnings_week,security_return_lag,earnings_week_lag,market_return
Date,,,,,
2011-01-03,NaN,False,NaN,NaN,NaN
2011-01-10,NaN,False,NaN,False,NaN
2011-01-18,-0.019984,True,NaN,False,-0.007219
2011-01-24,-0.017876,False,-0.019984,True,-0.005076
2011-01-31,0.016486,False,-0.017876,False,0.026501


In [7]:
def calc_beta(df, include_earnings=True):
    if not include_earnings:
        df = df.loc[(df['earnings_week'] == False)]
        df = df.loc[(df['earnings_week_lag'] == False)]
    model = sm.ols(formula='security_return ~ market_return + security_return_lag', data=df).fit()
    
    return model



In [ ]:
calc_beta(make_analysis_df('GOOG', stocks, market), include_earnings = False).params

In [ ]:
calc_beta(make_analysis_df('GOOG', stocks, market), include_earnings = True).params

In [9]:
def analyze_stocks(stocks, market, uni=universe_stocks):
    uni['w_e_alpha'] = np.nan
    uni['w_e_beta_market'] = np.nan
    uni['w_e_beta_sec_lag'] = np.nan
    uni['wo_e_alpha'] = np.nan
    uni['wo_e_beta_market'] = np.nan
    uni['wo_e_beta_sec_lag'] = np.nan
    
    for tic in stocks.keys():
        w_model = calc_beta(make_analysis_df(tic, stocks, market), include_earnings = True)
        uni['w_e_alpha'] = w_model.params[0]
        uni['w_e_beta_market'] = w_model.params[1]
        uni['w_e_beta_sec_lag'] = w_model.params[2]
        wo_model = calc_beta(make_analysis_df(tic, stocks, market), include_earnings = True)
        uni['wo_e_alpha'] = wo_model.params[0]
        uni['wo_e_beta_market'] = wo_model.params[1]
        uni['wo_e_beta_sec_lag'] = wo_model.params[2]
    
    return uni

results = analyze_stocks(stocks, market)

In [11]:
results.to_csv('first_pass_results.csv')